# EPL Player Stats 2024-25: xG Overperformance Analysis

This notebook analyzes Premier League player performance data focusing on Expected Goals (xG) overperformance.

## Goal
Identify the top 10 players who are scoring more goals than their xG suggests - the most clinical finishers.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

## Load Data

In [ ]:
# Load the EPL player stats dataset
df = pd.read_csv('../data/epl_player_stats_24_25.csv')

# Display basic info
print(f'Dataset shape: {df.shape}')
df.head()

## Calculate xG Overperformance

Formula: **Clinical Finishing = Actual Goals - Expected Goals (xG)**

In [ ]:
# TODO: Calculate xG overperformance
# df['xg_overperformance'] = df['goals'] - df['xg']

## Analysis & Visualization

Top 10 most clinical finishers in EPL 2024-25

In [ ]:
# TODO: Create visualizations
# 1. Bar chart of top 10 xG overperformers
# 2. Scatter plot: xG vs Actual Goals
# 3. Save plots to ../visuals/